In [1]:
#!/usr/bin/env python
import sys
import tensorflow as tf
import keras as keras

import matplotlib.pyplot as plt
import numpy as np

# set print options in jupyter
np.set_printoptions(precision=2, suppress=True)
np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(linewidth=250)

# load dataset 
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = tf.pad(x_train, [[0, 0], [2, 2], [2, 2]]) / 255
x_test  = tf.pad(x_test , [[0, 0], [2, 2], [2, 2]]) / 255

x_train = tf.expand_dims(x_train, axis = 3, name = None)
x_test  = tf.expand_dims(x_test , axis = 3, name = None)

x_val = x_train[-2000 : , : , : , :]
y_val = y_train[-2000 :]

x_train = x_train[:-2000, : , : , :]
y_train = y_train[:-2000]

# load pre-trained model
my_model = keras.models.load_model("./model")

#print model summary
print(my_model.summary())



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (None, 28, 28, 6)         156       
                                                                 
 pool1 (AveragePooling2D)    (None, 14, 14, 6)         0         
                                                                 
 activation (Activation)     (None, 14, 14, 6)         0         
                                                                 
 conv2 (Conv2D)              (None, 10, 10, 16)        2416      
                                                                 
 pool2 (AveragePooling2D)    (None, 5, 5, 16)          0         
                                                                 
 activation_1 (Activation)   (None, 5, 5, 16)          0         
                                                                 
 conv3 (Conv2D)              (None, 1, 1, 120)         4

In [2]:
# image : 7
test_image = x_test[0]

# for model input 
expanded_image = tf.expand_dims(test_image, axis=0)

# for get weight and bias
weight_bias_dict = {}
layer_list = []
for layer in my_model.layers : 
    layer_list.append(layer.name)
    if 'pool' in layer.name or 'flatten' in layer.name or 'activation' in layer.name:
        continue
    weight_bias_dict[layer.name] = layer.get_weights()

In [3]:
# weight_bias_dict['conv1']

In [4]:
print("Components of LeNet-5")
print("=====================")
for layer in layer_list :
    print(layer)
print("=====================\n")

print("Layers shape which have weights and bias")
print("=============================================================================================")
for key, value in weight_bias_dict.items() :
    print(f"layer name : {key}\t weight shape : {value[0].shape}\tbias shape : {value[1].shape}" .format(key=key, value=value) )
print("=============================================================================================\n")


Components of LeNet-5
conv1
pool1
activation
conv2
pool2
activation_1
conv3
flatten
full1
full2

Layers shape which have weights and bias
layer name : conv1	 weight shape : (5, 5, 1, 6)	bias shape : (6,)
layer name : conv2	 weight shape : (5, 5, 6, 16)	bias shape : (16,)
layer name : conv3	 weight shape : (5, 5, 16, 120)	bias shape : (120,)
layer name : full1	 weight shape : (120, 84)	bias shape : (84,)
layer name : full2	 weight shape : (84, 10)	bias shape : (10,)



In [5]:
# handling weights of convolution layers to make a good look
for key, value in weight_bias_dict.items() :
    if 'conv' in key :
        tp_weight = tf.transpose(value[0])
        weight = tf.transpose(tp_weight, [0, 1, 3, 2])
        value[0] = tf.squeeze(weight)
    if 'full' in key :
        tp_weight = tf.transpose(value[0])
        value[0] = tf.squeeze(tp_weight)

print("After handling the weights of convolution layers")
print("=============================================================================================")
for key, value in weight_bias_dict.items() :
    print(f"layer name : {key}\t weight shape : {value[0].shape}\tbias shape : {value[1].shape}" .format(key=key, value=value) )
print("=============================================================================================\n")


After handling the weights of convolution layers
layer name : conv1	 weight shape : (6, 5, 5)	bias shape : (6,)
layer name : conv2	 weight shape : (16, 6, 5, 5)	bias shape : (16,)
layer name : conv3	 weight shape : (120, 16, 5, 5)	bias shape : (120,)
layer name : full1	 weight shape : (84, 120)	bias shape : (84,)
layer name : full2	 weight shape : (10, 84)	bias shape : (10,)



In [6]:
output_dict = {}

for layer in my_model.layers :
    temp_model = keras.Model(inputs=my_model.input, outputs = my_model.get_layer(layer.name).output)
    output_dict[layer.name + '_output'] = temp_model(expanded_image)

print("Each layer output shape")
print("=============================================================================================")
for key, value in output_dict.items() :
    print(f"output name : {key}\t output shape : {value.shape}\t" .format(key=key, value=value))
print("=============================================================================================\n")


    

Each layer output shape
output name : conv1_output	 output shape : (1, 28, 28, 6)	
output name : pool1_output	 output shape : (1, 14, 14, 6)	
output name : activation_output	 output shape : (1, 14, 14, 6)	
output name : conv2_output	 output shape : (1, 10, 10, 16)	
output name : pool2_output	 output shape : (1, 5, 5, 16)	
output name : activation_1_output	 output shape : (1, 5, 5, 16)	
output name : conv3_output	 output shape : (1, 1, 1, 120)	
output name : flatten_output	 output shape : (1, 120)	
output name : full1_output	 output shape : (1, 84)	
output name : full2_output	 output shape : (1, 10)	



In [7]:
# handling outputs of layers to make a good look
for key, value in output_dict.items() :
    # for 4-dimension output 
    if 'conv' in key or 'pool' in key :
        tp_output = tf.transpose(value)
        tp_tp_output = tf.transpose(tp_output, [0, 2, 1, 3])
        output_dict[key] = tf.squeeze(tp_tp_output)
        continue
    # for 2-dimension output
    output_dict[key] = tf.squeeze(value)


In [8]:

print("After handling the output")
print("=============================================================================================")
for key, value in output_dict.items() :
    print(f"output name : {key}\t output shape : {value.shape}\t" .format(key=key, value=value))
print("=============================================================================================\n")

After handling the output
output name : conv1_output	 output shape : (6, 28, 28)	
output name : pool1_output	 output shape : (6, 14, 14)	
output name : activation_output	 output shape : (14, 14, 6)	
output name : conv2_output	 output shape : (16, 10, 10)	
output name : pool2_output	 output shape : (16, 5, 5)	
output name : activation_1_output	 output shape : (5, 5, 16)	
output name : conv3_output	 output shape : (120,)	
output name : flatten_output	 output shape : (120,)	
output name : full1_output	 output shape : (84,)	
output name : full2_output	 output shape : (10,)	



In [9]:
# print image
# tf.squeeze(test_image)

In [10]:
# print conv1 output 
# output_dict['conv1_output']

In [11]:
# print conv1 weight
# weight_bias_dict['conv1'][0]


In [13]:
# print conv1 bias
# print(weight_bias_dict['conv1'][1].shape)
# print(weight_bias_dict['conv1'][1])

In [14]:
# print pool1 output
# print(output_dict['pool1_output'])

In [15]:
# print conv2 output
# output_dict['conv2_output']

In [16]:
# print conv2 weight
# weight_bias_dict['conv2'][0]

In [17]:
# print conv2 bias
# print(weight_bias_dict['conv2'][1].shape)
# print(weight_bias_dict['conv2'][1])

In [18]:
# print pool2 output
# output_dict['pool2_output']

In [19]:
# print conv3 output
# output_dict['conv3_output']

In [20]:
# print conv3 weight
# weight_bias_dict['conv3'][0]

In [21]:
# print conv3 bias
# print(weight_bias_dict['conv3'][1].shape)
# print(weight_bias_dict['conv3'][1])

In [22]:
# print flatten output
# output_dict['flatten_output']

In [23]:
# print full1 output
# output_dict['full1_output']

In [24]:
# print full1 weight
# print(weight_bias_dict['full1'][0])
# print(weight_bias_dict['full1'][0].shape)

In [25]:
# print full2 bias
# print(weight_bias_dict['full1'][1])
# print(weight_bias_dict['full1'][1].shape)

In [26]:
# print full2 output
# output_dict['full2_output']

In [27]:
# print full2 weight
# print(weight_bias_dict['full2'][0])
# print(weight_bias_dict['full2'][0].shape)


In [28]:
# print full2 bias
# print(weight_bias_dict['full2'][1])
# print(weight_bias_dict['full2'][1].shape)

In [31]:
import os
import shutil
import random as rd

# shutil.rmtree("./data")

# os.mkdir("../data")
os.makedirs("../data/weight_origin", exist_ok=True)
os.makedirs("../data/bias_origin", exist_ok=True)
os.makedirs("../data/input", exist_ok=True)

# save weight & bias with txt
for key, value in weight_bias_dict.items() :
    value[0] = tf.reshape(value[0], [-1])
    
for key, value in weight_bias_dict.items() :
    np.savetxt('../data/weight_origin/' + key + '_weight_origin.data', value[0], fmt="%.10f")
    np.savetxt('../data/bias_origin/' + key + '_bias_origin.data'  , value[1], fmt="%.10f")